In [3]:
import sys
sys.path.append("../")

In [4]:
from CurvyCUSIPs.CurveDataFetcher import CurveDataFetcher
from CurvyCUSIPs.S490Swaps import S490Swaps
from CurvyCUSIPs.S490SwaptionsMarker import S490SwaptionsMarker  
from CurvyCUSIPs.utils.dtcc_swaps_utils import tenor_to_years, datetime_to_ql_date

from datetime import datetime
import QuantLib as ql
import pandas as pd
import numpy as np
import scipy as sp 

import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
plt.style.use('ggplot')
params = {'legend.fontsize': 'x-large',
        'figure.figsize': (18, 10),
        'axes.labelsize': 'x-large',
        'axes.titlesize':'x-large',
        'xtick.labelsize':'x-large',
        'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)

import nest_asyncio
nest_asyncio.apply()

%load_ext autoreload
%autoreload 2

In [5]:
curve_data_fetcher = CurveDataFetcher()
s490_swaps = S490Swaps(s490_curve_db_path=r"..\db\nyclose_sofr_ois", error_verbose=True)
s490_swaptions = S490SwaptionsMarker(
    s490_swaps,
    s490_vol_cube_markings_db_path=r"..\db\yield_curve_monkey_vol_cube_markings_db",
    s490_atm_vol_grird_markings_db_path=r"..\db\yield_curve_monkey_atm_vol_grid_markings_db",
)

..\db\nyclose_sofr_ois is behind --- cd into 'scripts' and run 'update_sofr_ois_db.py' to update --- most recent date in db: 2024-12-20 00:00:00
..\db\yield_curve_monkey_vol_cube_markings_db is behind --- cd into 'scripts' and run 'update_sofr_ois_db.py' to update --- most recent date in db: 2024-11-07 00:00:00
..\db\yield_curve_monkey_atm_vol_grid_markings_db is behind --- cd into 'scripts' and run 'update_sofr_ois_db.py' to update --- most recent date in db: 2024-11-07 00:00:00


In [6]:
as_of_date = datetime(2024, 12, 5)

sofr_ois_swaption_time_and_sales_dict, _, _ = s490_swaptions.create_s490_swaption_time_and_sales(
    start_date=as_of_date, 
    end_date=as_of_date, 
    data_fetcher=curve_data_fetcher, 
    model="normal"
    # model="lognormal"
)

df = sofr_ois_swaption_time_and_sales_dict[as_of_date]
display(df)
(df["Option Tenor"] + "x" + df["Underlying Tenor"]).drop_duplicates()

Fitting Swaption Trades...:   0%|          | 0/1 [00:00<?, ?it/s]2024-12-24 23:05:33,206 - CurvyCUSIPs.S490Swaps - ERROR - Error at 51 2024-12-05 00:00:00 - 1Mx10Y: root not bracketed: f[0,5] -> [2.744971e-03,4.806799e+00]
2024-12-24 23:05:33,823 - CurvyCUSIPs.S490Swaps - ERROR - Error at 95 2024-12-05 00:00:00 - 6Mx10Y: root not bracketed: f[0,5] -> [9.669927e-03,1.148994e+01]
Fitting Swaption Trades...: 100%|██████████| 1/1 [00:01<00:00,  1.31s/it]
2024-12-24 23:05:34,151 - CurvyCUSIPs.S490Swaps - ERROR - Errors Count: 2


,Event timestamp,Execution Timestamp,Effective Date,Expiration Date,Maturity date of the underlier,Fwd,Option Tenor,Underlying Tenor,Strike Price,Option Premium Amount,...,Option Premium per Notional,Direction,Style,Unique Product Identifier,UPI FISN,UPI Underlier Name,ATMF,OTM,IV,IV bp/day
0,2024-12-05 14:25:11,2024-12-05 14:25:11,2024-12-05,2025-06-05,2055-06-09,0D,6M,30Y,0.038500,4.592500e+05,...,0.008350,buyer,payer,QZZGWPNBF5R3,NA/O Call Epn OIS USD,NA/Swap OIS USD,0.033621,48.786562,58.927639,3.712092
1,2024-12-05 15:04:45,2024-12-05 15:04:45,2024-12-05,2025-02-06,2035-02-10,0D,2M,10Y,0.037110,5.355009e+05,...,0.002550,buyer,receiver,QZNLQ8T0N0SX,NA/O P Epn OIS USD,NA/Swap OIS USD,0.036229,8.807935,23.803867,1.499503
2,2024-12-05 15:06:49,2024-12-05 15:06:49,2024-12-05,2025-02-06,2032-02-10,0D,2M,7Y,0.042025,7.095000e+05,...,0.001650,buyer,payer,QZZGWPNBF5R3,NA/O Call Epn OIS USD,NA/Swap OIS USD,0.036147,58.783854,116.580689,7.343893
3,2024-12-05 15:09:43,2024-12-05 15:09:43,2024-12-05,2034-12-05,2044-12-07,0D,10Y,10Y,0.037700,1.745000e+07,...,0.174500,buyer,payer,QZBK1N7NJ0VF,NA/O Call Epn OIS USD,NA/Swap OIS USD,0.036647,10.531517,242.279060,15.262146
4,2024-12-05 15:09:43,2024-12-05 15:09:43,2024-12-05,2026-12-07,2036-12-09,0D,2Y,10Y,0.037000,3.628000e+06,...,0.090700,buyer,payer,QZBK1N7NJ0VF,NA/O Call Epn OIS USD,NA/Swap OIS USD,0.036120,8.796781,210.736236,13.275135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,2024-12-05 21:30:23,2024-12-05 21:30:23,2024-12-05,2028-12-05,2029-12-07,0D,4Y,12M,0.035610,4.880995e+06,...,0.008135,buyer,receiver,QZZGWPNBF5R3,NA/O P Epn OIS USD,NA/Swap OIS USD,0.035127,4.826513,120.756751,7.606960
90,2024-12-05 21:35:59,2024-12-05 21:35:59,2024-12-05,2025-12-05,2035-12-09,0D,12M,10Y,0.036500,3.641000e+06,...,0.033100,buyer,payer,QZNLQ8T0N0SX,NA/O Call Epn OIS USD,NA/Swap OIS USD,0.036036,4.638766,102.549830,6.460032
91,2024-12-05 21:35:59,2024-12-05 21:35:59,2024-12-05,2025-12-05,2035-12-09,0D,12M,10Y,0.036500,3.641000e+06,...,0.033100,buyer,receiver,QZNLQ8T0N0SX,NA/O P Epn OIS USD,NA/Swap OIS USD,0.036036,4.638766,102.549830,6.460032
92,2024-12-05 21:36:26,2024-12-05 21:36:26,2024-12-05,2025-12-05,2055-12-09,0D,12M,30Y,0.034860,3.265000e+06,...,0.065300,buyer,payer,QZNLQ8T0N0SX,NA/O Call Epn OIS USD,NA/Swap OIS USD,0.033340,15.199283,93.315155,5.878302


0       6Mx30Y
1       2Mx10Y
2        2Mx7Y
3      10Yx10Y
4       2Yx10Y
5       3Yx10Y
7        3Yx3Y
9        1Mx2Y
10     15Dx10Y
11     22Dx30Y
12      1Mx30Y
13      3Mx12M
14      2Mx30Y
16     188Dx2Y
18       2Yx5Y
20    216Dx12M
23     18Dx12M
24      6Mx10Y
26      3Mx10Y
28      1Mx10Y
30       1Mx5Y
52     12Mx10Y
54      12Mx5Y
56      7Yx20Y
58      4Yx20Y
60      5Yx30Y
62      4Yx30Y
64      1Mx12M
70      3Mx30Y
76      6Mx12M
86       4Yx2Y
88      4Yx12M
92     12Mx30Y
dtype: object

### Inject Dark Mode:

In [7]:
%%html
<style>
/*overwrite hard coded write background by vscode for ipywidges */
.cell-output-ipywidget-background {
   background-color: transparent !important;
}

/*set widget foreground text and color of interactive widget to vs dark theme color */
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}
</style>

In [27]:
hardcoded_swaption = {
    "Event timestamp": as_of_date,
    "Option Tenor": "10Y",
    "Underlying Tenor": "1Y",
    "Strike Price": 4.046216 / 100,
    "ATMF": 4.046216 / 100,
    "IV": 5.91 * np.sqrt(252),
    "IV bp/day": 5.91,
} 

In [ ]:
s490_swaptions.create_sabr_smile_interactive(
    swaption_time_and_sales=sofr_ois_swaption_time_and_sales_dict[as_of_date],
    option_tenor="12M",
    underlying_tenor="10Y",
    initial_beta=0.5,
    show_trades=True,
    scale_by_notional=False,
    model="normal",
    year_day_count=252,
    ploty_height=1250,
)

,Event timestamp,Direction,Style,Notional Amount,Strike Price,OTM,Option Premium per Notional,IV,IV bp/day,ATMF,UPI FISN,UPI Underlier Name
0,2024-12-05 16:15:49,buyer,payer,75000000.0,0.036725,6.888766,0.0331,103.962826,6.549042,0.036036,NA/O Call Epn OIS USD,NA/Swap OIS USD
1,2024-12-05 16:15:49,buyer,receiver,75000000.0,0.036725,6.888766,0.0331,103.962826,6.549042,0.036036,NA/O P Epn OIS USD,NA/Swap OIS USD
2,2024-12-05 21:35:59,buyer,payer,55000000.0,0.036500,4.638766,0.0331,102.549830,6.460032,0.036036,NA/O Call Epn OIS USD,NA/Swap OIS USD
3,2024-12-05 21:35:59,buyer,receiver,55000000.0,0.036500,4.638766,0.0331,102.549830,6.460032,0.036036,NA/O P Epn OIS USD,NA/Swap OIS USD


Output()

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'SABR Smile',
              'type': 'scatter',
              'uid': '66b22654-a66b-49a9-af9c-5f1fc014e35a',
              'x': array([-200, -150, -100,  -50,  -25,    0,   25,   50,  100,  150,  200]),
              'y': [85.65093692586986, 90.67320732915549, 95.12130384773448,
                    99.15611990956425, 101.05048395779794, 102.87504021257845,
                    104.63697483558133, 106.34231959816174, 109.6029964239434,
                    112.69008885099392, 115.62865757595206]},
             {'hovertemplate': ('Strike Offset: %{x:.3f} bps<br' ... 'tyle: payer<br><extra></extra>'),
              'marker': {'color': 'green', 'size': 8},
              'mode': 'markers',
              'name': '16:15:49-payer buyer',
              'type': 'scatter',
              'uid': 'ad445009-db59-448f-a8d2-c0352a5873b7',
              'x': [6.888766335547622],
              'y': [103.96282585031136]},
             {'hove

Output()